This notebook pulls nightlight data form the engine and save it to a raster file.

In [1]:
import ee
ee.Initialize()

# Study Area
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
chile  = ee.Feature(countries.filter(ee.Filter.eq('country_na','Chile')).first()).geometry()
circle = ee.Geometry.Point(-72.388,-38.905).buffer(300000)
area = chile.intersection(circle)

# Importing Shape of Mercy Titles 
shape = ee.FeatureCollection('users/fjordancolzani/mt_shape')

# Import Nightlight Series
nl = ee.ImageCollection('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS').filterBounds(area).sort('DATE_ACQUIRED')
nl92 = ee.Image(nl.first()).select(1)  

In [7]:
# Average Nightlight in radios 25 and 50 and 100 KM
for r in [100000]:
    shape_b = shape.map(lambda x: x.centroid().buffer(r))
    lights = nl92.reduceRegions(collection=shape_b,reducer = ee.Reducer.mean())
    des = 'lights92_'+str(r/1000)+'km'
    task1 =  ee.batch.Export.table.toDrive(collection=lights,description=des,
                                       folder='EarthEngineResults')
    task1.start()

